In [1]:
from lis import evaluate

In [2]:
p1 = "(begin (define r 10) (* pi (* r r)))"
evaluate(p1)

314.1592653589793

In [3]:
p2 = ("(car (list 1 2 3 4))")
evaluate(p2)

1

In [4]:
p3 = """
(begin 
  (define fact (lambda (n) (if (<= n 1) 1 (* n (fact (- n 1))))))
  (fact 100)
)
"""
%timeit evaluate(p3)
evaluate(p3)

1.29 ms ± 4.99 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000

In [5]:
fact = lambda n: 1 if n<=1 else n * fact(n-1)
%timeit fact(100)
fact(100)


13.2 µs ± 174 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000

In [6]:
p4 = """
(begin 
  (define twice (lambda (x) (* 2 x)))
  (twice 10)
)
"""

evaluate(p4)

20

In [7]:
p5 = """
(begin
  (define fib (lambda (n) (if (< n 2) 1 (+ (fib (- n 1)) (fib (- n 2))))))
  (fib 10)
)
"""
%timeit evaluate(p5)
evaluate(p5)

1.87 ms ± 8.49 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


89

In [8]:
fib = lambda n: 1 if n < 2 else fib(n - 1) + fib (n - 2)
%timeit fib(10)
fib(10)

13.7 µs ± 144 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


89

In [9]:
p6 = """
(begin
  (define move (lambda (n from to spare)
    (if 
      (= 0 n) "Done_Moving"
      (begin
        (move (- n 1) from spare to)
        (print (list "Move" n "from" from "to" to))
        (move (- n 1) spare to from)
      )
    ))
)
  (move 3 "One" "Three" "Two")
)
"""

evaluate(p6)

['"Move"', 1, '"from"', '"One"', '"to"', '"Three"']
['"Move"', 2, '"from"', '"One"', '"to"', '"Two"']
['"Move"', 1, '"from"', '"Three"', '"to"', '"Two"']
['"Move"', 3, '"from"', '"One"', '"to"', '"Three"']
['"Move"', 1, '"from"', '"Two"', '"to"', '"One"']
['"Move"', 2, '"from"', '"Two"', '"to"', '"Three"']
['"Move"', 1, '"from"', '"One"', '"to"', '"Three"']


'"Done_Moving"'

In [10]:
p7 ="""
(begin
  (define threshold 1e-6)
  (define dx 1e-3)
  
  (define newton (lambda (f guess)
    (fixed-point (lambda (x) (- x (/ (f x) ((derive f threshold) x)))) guess)))

  (define derive (lambda (f dx)
    (lambda (x) (/ (- (f (+ x dx)) (f x)) dx))))

  (define close-enough? (lambda (x y)
    (< (abs (- x y)) threshold)))
    
  (define fixed-point (lambda (f n)
    (if (close-enough? n (f n))
        n
        (fixed-point f (f n)))))

  (define jazr (lambda (y)
    (newton (lambda (x) (- y (* x x)))  1.0)))

  (jazr 2)
)
"""

evaluate(p7)

1.4142135623754424

In [11]:
p8 = """
(begin
  (define deriv (lambda (exp var)
    (if (constant? exp var) 0
        (if  (same-var? exp var) 1
             (if (sum? exp) (make-sum (deriv (a1 exp) var) (deriv (a2 exp) var))
                 (if (product? exp) (make-sum
                           (make-product (m1 exp) (deriv (m2 exp) var))
                           (make-product (m2 exp) (deriv (m1 exp) var)))
                     "ERROR"))))))
         
  (define atomic? (lambda (exp) (not (list? exp))))

  (define constant? (lambda (exp var) (and (atomic? exp) (not (equal? exp var)))))

  (define same-var? (lambda (exp var) (and (atomic? exp) (equal? exp var))))

  (define sum? (lambda (exp) (and (list? exp) (equal? (car exp) (quote +)))))

  (define product? (lambda (exp) (and (list? exp) (equal? (car exp) (quote *)))))

  (define make-sum (lambda (a1 a2)
    (if (equal? a1 0) a2
        (if (equal? a2 0) a1
            (list (quote +) a1 a2)))))
  
  (define make-product (lambda (m1 m2)
    (if (equal? m1 1) m2
        (if (equal? m2 1) m1
            (if (or (equal? m1 0) (equal? m2 0)) 0
                (list (quote *) m1 m2))))))

  (define a1 (lambda (l) (car (cdr l))))

  (define a2 (lambda (l) (car (cdr (cdr l)))))

  (define m1 a1)

  (define m2 a2)

  (deriv (quote (+ (* x x) (* 2 x))) (quote x))
)
"""
evaluate(p8)

['+', ['+', 'x', 'x'], 2]